# 要約 
このJupyterノートブックは、「20の質問」ゲームに挑戦するための言語モデルの開発に関連しており、特定のキーワードを当てるための質問と推測を行うエージェントを構築しています。以下に、ノートブックの主な内容を要約します。

### 問題の取り組み
ノートブックは、言語モデルを使用して、質問応答形式で特定のキーワードを推測するAIエージェントを構築することを目的としています。特に、ユーザーが設定したカテゴリや特定の特徴に基づいて、質問を生成し、それに対する応答を取得して推測を行います。このプロジェクトは、Kaggleの「LLM 20 Questions」コンペティションに参加するために設計されています。

### 使用されている手法とライブラリ
1. **データ処理**:
   - `pandas`: データ操作を行い、キーワードデータを管理します。
   - JSON形式のキーワードデータを辞書として読み込み、DataFrame形式に変換しています。

2. **ビジュアライゼーション**:
   - `matplotlib`: クエスチョンやアンサーを視覚化するために使用され、描画機能を実装しています。

3. **モデルの構築**:
   - `transformers`: Hugging FaceのTransformersライブラリからT5モデルを用いることが考慮されています。また、別の言語モデル「Llama-3-Smaug-8B」を使用してYes/No質問の応答を生成しています。

4. **対話システムの実装**:
   - 多層的な質問生成や推測アルゴリズムを通じて、エージェントは、質問に対する応答を生成するほか、さらに次の質問を決定します。これには、質問の種類（例えば、地名や国名、文化財など）や特徴（大陸や文字の最初など）に基づく処理があります。

5. **リソース管理**:
   - `gc`および`torch.cuda`: メモリ管理を行い、GPUリソースのクリアを実施しています。

### 結論
このノートブックは「20の質問」ゲームのAIエージェントを開発するための基盤を提供しており、特定の対象を当てるための戦略的な質問を自動的に生成し、言語モデルによって得られた情報を効果的に利用します。また、モデルとトークナイザーの読み込み、応答処理のロジック、エラーハンドリングについても詳細に記載されており、実用的なプロジェクトとして集約されています。

---


# 用語概説 
以下は、Jupyter Notebookの内容に基づいて、機械学習・深層学習の初心者がつまずきそうな専門用語の簡単な解説です。初心者なレベルですが、少しマイナーな用語や特有のドメイン知識に焦点を当てています。

1. **immutable_dict**: イミュータブル（不変）な辞書を提供するデータ構造で、作成後に内容を変更できません。この特性は、データ整合性を保つために役立ちます。

2. **sentencepiece**: Googleが開発したオープンソースのトークン化ツールです。主に、ニューラルネットワークの入力としてテキストを効果的に処理するために使用されます。モデリングの際にサブワード単位での処理を支援します。

3. **T5Tokenizer / T5ForConditionalGeneration**: T5（Text-to-Text Transfer Transformer）モデル用のトークナイザーおよび条件付き生成モデルです。テキストを生成や要約、翻訳など様々なタスクに利用できる柔軟性があります。

4. **pip install -U -t**: Pythonのパッケージマネージャであるpipを使って、指定したフォルダにパッケージをインストールするコマンド。`-U`オプションは既存のパッケージをアップグレードすることを意味します。`-t`オプションはインストール先のディレクトリを指定します。

5. **maximum new tokens**: モデルが生成する新しいトークンの最大数を指定するパラメータで、これにより生成された応答の長さを制限します。

6. **torch.no_grad()**: PyTorchにおいて、勾配計算を無効にするための文脈マネージャです。推論時にメモリ使用量を削減し、計算時間を短縮します。

7. **torch.cuda.amp.autocast()**: 自動混合精度（Automatic Mixed Precision）を有効にするための文脈マネージャで、計算を半精度で行うことにより性能を向上させることができます。リソースの効率を良くし、深層学習のトレーニングや推論を加速します。

8. **get_yes_no()関数**: 与えられた質問に対して「はい」または「いいえ」で応答するための関数です。キーワードやその属性に基づいてプロンプトを組み立て、モデルに入力します。

9. **Observationクラス**: エージェントのターンに関する情報（質問、回答、キーワードなど）を管理するデータ構造を定義するクラスで、システムの状態を保持します。

10. **garbage collection (gc)**: 使用されなくなったメモリリソースを自動的に解放するプロセスです。メモリリークを防ぎ、アプリケーションのパフォーマンスを向上させます。

11. **Assertions**: コード内の条件が本当に真であるかどうかを確認するためのテスト機能。エラーが発生した場合には、デバッグの助けとなります。

これらの説明は、初心者がノートブックを理解するのに役立つ関連用語をより明確にするためのものです。

---


In [ ]:
# このPython 3環境には、多くの便利な分析ライブラリがインストールされています
# これは、kaggle/pythonのDockerイメージに基づいています: https://github.com/kaggle/docker-python
# 例えば、いくつかの便利なパッケージを読み込むことができます

import numpy as np # 線形代数を扱うためのライブラリ
import pandas as pd # データ処理やCSVファイルの入出力を行うライブラリ（例: pd.read_csv）

# 入力データファイルは読み取り専用の "../input/" ディレクトリにあります
# 例えば、これを実行すると（実行ボタンをクリックするかShift+Enterを押すと）入力ディレクトリ内のすべてのファイルが一覧表示されます

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename)) # 各ファイルのパスを表示します

# 現在のディレクトリ (/kaggle/working/) に最大20GBまで書き込むことができ、そのデータは "Save & Run All" を使用してバージョンを作成する際に出力として保存されます
# また、一時ファイルを /kaggle/temp/ にも書き込むことができますが、現在のセッションの外には保存されません

In [ ]:
%%bash
cd /kaggle/working # 作業ディレクトリに移動
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece # immutabledictとsentencepieceを作業ディレクトリ内の特定のライブラリフォルダにインストールします
git clone https://github.com/google/gemma_pytorch.git > /dev/null # gemma_pytorchリポジトリをクローンします（出力は非表示）
mkdir /kaggle/working/submission/lib/gemma/ # gemma用のディレクトリを作成します
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/ # クローンしたファイルを指定のディレクトリに移動します

In [ ]:
import json # JSONデータを扱うためのライブラリをインポート
import os # OSに関する機能を提供するライブラリをインポート
import pandas as pd # データ操作のためのpandasライブラリをインポート
import random # ランダムな値を生成するためのライブラリをインポート
import string # 文字列操作のためのライブラリをインポート
import torch # PyTorchライブラリをインポート（深層学習用）

from os import path # OSのpath機能をインポート
from pathlib import Path # パス操作のためのPathクラスをインポート
from random import choice # ランダムに選択するためのchoice関数をインポート
from string import Template # テンプレート文字列操作のためのTemplateクラスをインポート
from transformers import T5Tokenizer, T5ForConditionalGeneration # T5モデル用のトークナイザーとモデルをインポート

# キーワードのJSONデータを定義
KEYWORDS_JSON = """
[
  {
    "category": "country", # カテゴリ: 国
    "words": [
      {
        "keyword": "afghanistan", # キーワード: アフガニスタン
        "alts": [] # 代替単語: なし
      },
      {
        "keyword": "albania", # キーワード: アルバニア
        "alts": [] # 代替単語: なし
      },
      {
        "keyword": "algeria", # キーワード: アルジェリア
        "alts": [] # 代替単語: なし
      },
      ...
      {
        "keyword": "zimbabwe", # キーワード: ジンバブエ
        "alts": [] # 代替単語: なし
      }
    ]
  },
  {
    "category": "city", # カテゴリ: 都市
    "words": [
      {
        "keyword": "amsterdam netherlands", # キーワード: アムステルダム, オランダ
        "alts": ["amsterdam", "amsterdam holland"] # 代替単語: アムステルダム, アムステルダム・ホラント
      },
      ...
      {
        "keyword": "zurich switzerland", # キーワード: チューリッヒ, スイス
        "alts": ["zurich"] # 代替単語: なし
      }
    ]
  },
  ...
]
""" # JSONの内容の終わり

In [ ]:
keywords_data = json.loads(KEYWORDS_JSON) # JSON形式のデータをPythonの辞書として読み込む

# DataFrameの行を格納するリストを作成
rows = []

# JSONの構造を行にフラット化する
for item in keywords_data:
    category = item["category"] # カテゴリを取得
    for word in item["words"]:
        rows.append({
            "keyword": word["keyword"], # キーワードを取得
            "category": category, # カテゴリを設定
            "continent": "",  # 大陸データがあれば追加可能
            "first_letter": word["keyword"][0].upper()  # 最初の文字を大文字に変換
        })

# DataFrameを作成
keywords = pd.DataFrame(rows) # フラット化したデータからDataFrameを作成
keywords.head() # DataFrameの最初の5行を表示

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def wrap_text(text, max_width, font_size):
    words = text.split() # テキストを単語に分割
    lines = [] # 行を格納するリスト
    line = "" # 行の初期化
    for word in words:
        test_line = f"{line} {word}".strip() # 次の行をテスト
        if len(test_line) * font_size > max_width and line: # 最大幅を超えたら
            lines.append(line) # 現在の行を保存
            line = word # 新しい行を開始
        else:
            line = test_line # 行を更新
    if line:
        lines.append(line) # 最後の行を保存
    return lines # 行を返す

def renderer(context):
    # コンテキストから必要な情報を取得
    act = context['act']
    agents = context['agents']
    environment = context['environment']
    frame = context['frame']
    height = context.get('height', 800) # 高さの取得
    interactive = context['interactive']
    is_interactive = context['is_interactive']
    parent = context['parent']
    step = context['step']
    update = context['update']
    width = context.get('width', 1200) # 幅の取得

    # 共通の寸法
    max_width = 1200
    max_height = 800
    canvas_size = min(height, width) # 最小のキャンバスサイズ
    unit = 8
    offset = canvas_size * 0.1 if canvas_size > 400 else unit / 2 # オフセットの計算
    cell_size = (canvas_size - offset * 2) / 3 # セルサイズの計算

    # 図と軸を作成
    fig, ax = plt.subplots(figsize=(width / 100, height / 100)) # 図のサイズを設定
    ax.set_xlim(0, width) # x軸の範囲を設定
    ax.set_ylim(0, height) # y軸の範囲を設定
    ax.invert_yaxis() # y軸の反転
    ax.set_axis_off() # 軸を非表示

    # 現在のステップの状態を取得
    try:
        steps = environment['steps']
        state = steps[step] # 現在のステップを取得
    except (IndexError, KeyError) as e:
        print(f"Error accessing environment steps: {e}") # エラーメッセージ
        return

    print("State:", state) # ステート情報を表示

    if not isinstance(state, dict):
        print("State is not properly defined.") # ステートが不正な場合のエラーメッセージ
        return

    # ステートから情報を抽出
    try:
        observation = state['observation']
        team1_index = len(observation['questions']) - 1
        team2_index = len(observation['questions']) - 1

        team1_question = observation['questions'][team1_index] if team1_index >= 0 else ""
        team2_question = observation['questions'][team2_index] if team2_index >= 0 else ""

        team1_answer = observation['answers'][team1_index] if len(observation['questions']) == len(observation['answers']) and team1_index >= 0 else ""
        team2_answer = observation['answers'][team2_index] if len(observation['questions']) == len(observation['answers']) and team2_index >= 0 else ""

        team1_guess = observation['guesses'][team1_index] if len(observation['questions']) == len(observation['guesses']) and team1_index >= 0 else ""
        team2_guess = observation['guesses'][team2_index] if len(observation['questions']) == len(observation['guesses']) and team2_index >= 0 else ""

        team1_reward = str(state['reward']) if state['reward'] != 0 else ""
        team2_reward = str(state['reward']) if state['reward'] != 0 else ""
    except (IndexError, KeyError) as e:
        print(f"Error extracting data from state: {e}") # エラーメッセージ
        return

    info = environment.get('info', {})
    team1_text = info.get('TeamNames', ['Team 1'])[0] # チーム1の名前を取得
    team2_text = info.get('TeamNames', ['Team 2'])[1] # チーム2の名前を取得

    # キャンバスパラメータ
    padding = 20
    row_width = (min(max_width, width) - padding * 3 - 100) / 2 # 行の幅を計算
    label_x = padding # ラベルのx座標
    team1_x = padding + 100 # チーム1のx座標
    team2_x = padding * 2 + row_width + 100 # チーム2のx座標
    line_height = 40 # 行の高さ
    label_y = 120 # ラベルのy座標
    question_y = 160 # 質問のy座標
    answer_y = 200 # 回答のy座標
    guess_y = 240 # 推測のy座標
    score_y = 280 # スコアのy座標

    # フォントサイズ
    font_size = 20 # フォントのサイズ

    # テキストを描画
    try:
        # キーワード行
        ax.text(label_x, line_height, f"Keyword: {observation['keyword']}", fontsize=font_size, color='#FFFFFF') # キーワードを描画

        # チーム行
        ax.text(team1_x, line_height, team1_text, fontsize=font_size, color='#FFFFFF') # チーム1の名前を描画
        ax.text(team2_x, line_height, team2_text, fontsize=font_size, color='#FFFFFF') # チーム2の名前を描画

        # 質問行
        ax.text(label_x, question_y, "Question:", fontsize=font_size, color='#FFFFFF') # 質問ラベルを描画
        wrapped_text1 = wrap_text(team1_question, row_width, font_size) # チーム1の質問をラップ
        wrapped_text2 = wrap_text(team2_question, row_width, font_size) # チーム2の質問をラップ
        for i, line in enumerate(wrapped_text1):
            ax.text(team1_x, question_y - i * line_height, line, fontsize=font_size, color='#FFFFFF') # チーム1のラップされた質問を描画
        for i, line in enumerate(wrapped_text2):
            ax.text(team2_x, question_y - i * line_height, line, fontsize=font_size, color='#FFFFFF') # チーム2のラップされた質問を描画

        # 回答行
        ax.text(label_x, answer_y, "Answer:", fontsize=font_size, color='#FFFFFF') # 回答ラベルを描画
        ax.text(team1_x, answer_y, team1_answer, fontsize=font_size, color='#FFFFFF') # チーム1の回答を描画
        ax.text(team2_x, answer_y, team2_answer, fontsize=font_size, color='#FFFFFF') # チーム2の回答を描画

        # 推測行
        ax.text(label_x, guess_y, "Guess:", fontsize=font_size, color='#FFFFFF') # 推測ラベルを描画
        ax.text(team1_x, guess_y, team1_guess, fontsize=font_size, color='#FFFFFF') # チーム1の推測を描画
        ax.text(team2_x, guess_y, team2_guess, fontsize=font_size, color='#FFFFFF') # チーム2の推測を描画

        # 報酬行
        ax.text(label_x, score_y, "Reward:", fontsize=font_size, color='#FFFFFF') # 報酬ラベルを描画
        ax.text(team1_x, score_y, team1_reward, fontsize=font_size, color='#FFFFFF') # チーム1の報酬を描画
        ax.text(team2_x, score_y, team2_reward, fontsize=font_size, color='#FFFFFF') # チーム2の報酬を描画

        plt.show() # グラフを表示
    except Exception as e:
        print(f"Error during rendering: {e}") # 描画中のエラーメッセージ

# テスト用のコンテキストの例
context = {
    'act': lambda x: print(f"Action: {x}"), # アクションを処理するための関数
    'agents': [],
    'environment': {
        'steps': [
            {'observation': {'questions': ['What is AI?', 'Define ML.'], 'answers': ['Artificial Intelligence', 'Machine Learning'], 'guesses': ['AI', 'ML'], 'keyword': 'AI'}, 'reward': 10}, # ステップデータ
        ],
        'info': {'TeamNames': ['Team Alpha', 'Team Beta']} # チーム名情報
    },
    'frame': None,
    'height': 800, # 高さ
    'interactive': True,
    'is_interactive': lambda: True,
    'parent': None,
    'step': 0,
    'update': lambda: None,
    'width': 1200 # 幅
}

renderer(context) # 描画関数を呼び出す

In [ ]:
import pandas as pd # pandasをインポート
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig # Transformersライブラリから必要なクラスをインポート
import torch # PyTorchライブラリをインポート
import numpy as np # NumPyライブラリをインポート
import gc # ガーベジコレクションを管理するためのライブラリをインポート
import os # OSに関する機能を提供するライブラリをインポート

# グローバル変数および設定
model_name = "abacusai/Llama-3-Smaug-8B"  # メモリの問題が発生する場合は小さいモデルを検討
VERBOSE = True  # デバッグ出力用にTrueに設定

# GPUサポートでモデルとトークナイザーをロードする関数
def load_model_and_tokenizer(model_name):
    try:
        config = AutoConfig.from_pretrained(model_name) # モデルの設定をロード
        model = AutoModelForCausalLM.from_pretrained( # 条件付き生成用のモデルをロード
            model_name,
            config=config,
            trust_remote_code=True, # リモートコードを信頼
            device_map="auto"  # 適切なデバイスにモデルを配置
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name) # トークナイザーをロード
        model.eval()  # モデルを評価モードに設定
        return model, tokenizer # モデルとトークナイザーを返す
    except Exception as e:
        if VERBOSE:
            print(f"Error loading model: {e}") # エラーメッセージ
        return None, None # エラーが発生した場合はNoneを返す

# Yes/No質問用にLLMと対話する関数
def get_yes_no(question, keyword, tokenizer, model):
    try:
        # プロンプトメッセージを構築
        if keyword in keywords.keyword.values: # キーワードが存在する場合
            row = keywords.loc[keywords.keyword == keyword].iloc[0] # キーワードに対応する行を取得
            category = row.category # カテゴリを取得
            continent = row.continent # 大陸情報を取得
            negate = { # 否定要素を定義
                "city": "It is not a country. It is not a landmark.",
                "country": "It is not a city. It is not a landmark.",
                "landmark": "It is not a city. It is not a country.",
            }
            prompt = f"We are playing 20 questions. The keyword is {keyword}. It is a {category}. {negate[category]} This word has first letter {keyword[0]}. This {category} is located in {continent}. {question}"
        else:
            prompt = f"We are playing 20 questions. The keyword is {keyword}. It is a thing. It is not a city. It is not a country. It is not a landmark. This word has first letter {keyword[0]}. {question}"

        messages = [
            {"role": "system", "content": "Answer yes or no to the following question and nothing else."}, # システムメッセージ
            {"role": "user", "content": prompt} # ユーザープロンプト
        ]

        # チャットテンプレートを適用し、トークン化
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to('cuda') # トークン化した入力をGPUに転送

        pad_token_id = tokenizer.pad_token_id # パディングトークンIDを取得
        if pad_token_id is None:
            pad_token_id = tokenizer.eos_token_id # Noneの場合はEOSトークンIDを使用

        # 混合精度を使用して応答を生成
        with torch.no_grad(), torch.cuda.amp.autocast():
            generated_ids = model.generate(
                model_inputs.input_ids,
                attention_mask=model_inputs.attention_mask,
                pad_token_id=pad_token_id,
                max_new_tokens=1 # 新しいトークンの最大数を設定
            )
        
        generated_ids = [
            output_ids[len(model_inputs.input_ids[0]):] for output_ids in generated_ids # 生成されたIDを抽出
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0] # 応答をデコード
        if not "yes" in response.lower(): # 応答が「yes」でなければ
            response = "no" # 「no」と返す
        else:
            response = "yes" # 「yes」と返す

        return response # 最終応答を返す
    
    except Exception as e:
        if VERBOSE:
            print(f"Error during agent execution: {e}") # エラーメッセージ
        return "no" # エラー時は「no」を返す
    
def agent_fn(obs, cfg): # エージェント関数
    global keywords # グローバル変数としてキーワードを参照

    # 応答変数の初期化
    response = "no"  # 応答のデフォルト値を設定
    
    try:
        # キーワードが正しくロードされていることを確認
        if keywords.empty: # キーワードが空の場合
            response = "Unable to process due to missing keywords." # エラーメッセージを設定
            return response

        # 初期化が必要な変数
        cat_guess = 0  # カテゴリ推測用カウンタ
        con_guess = 0  # 大陸推測用カウンタ
        let_guess = 0  # 最初の文字推測用カウンタ
        category_yes = [] # 「はい」回答のカテゴリリスト
        category_no = [] # 「いいえ」回答のカテゴリリスト
        continent_yes = [] # 「はい」回答の大陸リスト
        continent_no = [] # 「いいえ」回答の大陸リスト
        first_letter_yes = [] # 「はい」回答の最初の文字リスト
        first_letter_no = [] # 「いいえ」回答の最初の文字リスト
        extra_guess = None # 追加の推測

        categories = ["city", "country", "landmark"] # カテゴリのリスト
        continents = ["Europe", "Asia", "North America", "Africa", "South America", "Australia"] # 大陸のリスト

        if obs.turnType == "ask": # 問いかけの場合
            if (cat_guess < 3) and (len(category_yes) == 0): # カテゴリの推測
                response = f"Is the keyword the name of a {categories[cat_guess]}?" # 質問の構築
                cat_guess += 1 # カテゴリカウンタの増加
            elif (con_guess < 6) and (len(continent_yes) == 0): # 大陸の推測
                category = "place" # 場所として初期化
                if len(category_yes) == 1: 
                    category = category_yes[0] # 既存のカテゴリをセット
                response = f"Is the {category} located in {continents[con_guess]}?" # 質問の構築
                con_guess += 1 # 大陸カウンタの増加
            else:
                IDX = keywords.category.isin(category_yes) # カテゴリのインデックス
                IDX = IDX & (keywords.continent.isin(continent_yes)) # 大陸のインデックス
                first_letters = list(keywords.loc[IDX, "first_letter"].value_counts().index.values) # 最初の文字のリスト
                if let_guess < len(first_letters): # 最初の文字がリストにある場合
                    response = f"Does the keyword begin with the letter {first_letters[let_guess]}?" # 質問の構築
                else:
                    IDX = keywords.guess == 0 # 推測が初めての場合
                    if len(category_yes) > 0: IDX = IDX & (keywords.category.isin(category_yes)) # 「はい」回答対象カテゴリをフィルタ
                    if len(category_no) > 0: IDX = IDX & (~keywords.category.isin(category_no)) # 「いいえ」回答対象カテゴリをフィルタ
                    if len(continent_yes) > 0: IDX = IDX & (keywords.continent.isin(continent_yes)) # 「はい」回答対象大陸をフィルタ
                    if len(continent_no) > 0: IDX = IDX & (~keywords.continent.isin(continent_no)) # 「いいえ」回答対象大陸をフィルタ
                    if len(first_letter_yes) > 0: IDX = IDX & (keywords.first_letter.isin(first_letter_yes)) # 「はい」回答対象最初の文字をフィルタ
                    if len(first_letter_no) > 0: IDX = IDX & (~keywords.first_letter.isin(first_letter_no)) # 「いいえ」回答対象最初の文字をフィルタ

                    try:
                        guess = keywords.loc[IDX].sample(1).index.values[0] # 推測を抽出
                        keywords.loc[guess, 'guess'] = 1 # 推測マーク
                        response = keywords.loc[guess, "keyword"] # 推測結果を取得
                    except:
                        response = np.random.choice(keywords.keyword.values) # ランダムな推測を生成
                    extra_guess = response # 追加推測にセット
                    response = f"Is it {response}?" # 質問の構築
                let_guess += 1 # 最初の文字カウンタの増加

        elif obs.turnType == "guess": # 推測の場合
            category_yes = [] # 「はい」回答のカテゴリリストを初期化
            category_no = [] # 「いいえ」回答のカテゴリリストを初期化
            for k in range(cat_guess): # カテゴリ推測に対する回答を取得
                if obs.answers[k] == "yes":
                    category_yes.append(categories[k]) # 「はい」回答を追加
                else:
                    category_no.append(categories[k]) # 「いいえ」回答を追加
            if (cat_guess == 3) and (len(category_yes) == 0): # 全てのカテゴリが「いいえ」だった場合
                category_yes = ["city", "country", "landmark"] # すべてのカテゴリを対象にする
                category_no = [] # 「いいえ」リストをクリア

            continent_yes = [] # 大陸「はい」リストを初期化
            continent_no = [] # 大陸「いいえ」リストを初期化
            for k in range(con_guess): # 大陸推測に対する回答を取得
                if obs.answers[k + cat_guess] == "yes":
                    continent_yes.append(continents[k]) # 「はい」回答を追加
                else:
                    continent_no.append(continents[k]) # 「いいえ」回答を追加
            if (con_guess == 6) and (len(continent_yes) == 0): # 全ての大陸が「いいえ」だった場合
                continent_yes = ["Europe", "Asia", "North America", "Africa", "South America", "Australia"] # すべての大陸を対象にする
                continent_no = [] # 「いいえ」リストをクリア

            first_letter_yes = [] # 最初の文字「はい」リストを初期化
            first_letter_no = [] # 最初の文字「いいえ」リストを初期化
            for k in range(let_guess): # 最初の文字推測に対する回答を取得
                if k >= len(first_letters): continue # 文字数がオーバーしている場合、スキップ
                if obs.answers[k + cat_guess + con_guess] == "yes":
                    first_letter_yes.append(first_letters[k]) # 「はい」回答を追加
                else:
                    first_letter_no.append(first_letters[k]) # 「いいえ」回答を追加

            # フィルタリング条件を適用
            IDX = keywords.guess == 0
            if len(category_yes) > 0: IDX = IDX & (keywords.category.isin(category_yes))
            if len(category_no) > 0: IDX = IDX & (~keywords.category.isin(category_no))
            if len(continent_yes) > 0: IDX = IDX & (keywords.continent.isin(continent_yes))
            if len(continent_no) > 0: IDX = IDX & (~keywords.continent.isin(continent_no))
            if len(first_letter_yes) > 0: IDX = IDX & (keywords.first_letter.isin(first_letter_yes))
            if len(first_letter_no) > 0: IDX = IDX & (~keywords.first_letter.isin(first_letter_no))

            try:
                guess = keywords.loc[IDX].sample(1).index.values[0] # 推測を抽出
                keywords.loc[guess, 'guess'] = 1 # 推測マーク
                response = keywords.loc[guess, "keyword"] # 推測結果を取得
            except:
                response = np.random.choice(keywords.keyword.values) # ランダムな推測を生成

            if (let_guess > 0) and (let_guess >= len(first_letters)) and (obs.answers[-1] == "yes"): # 最後の回答が「はい」の場合
                response = extra_guess # 追加の推測を応答

        elif obs.turnType == "answer": # 回答の場合
            if obs.keyword.lower() in obs.questions[-1].lower(): # キーワードが質問に含まれている場合
                response = "yes" # 「はい」と応答
            else:
                response = get_yes_no(obs.questions[-1], obs.keyword, tokenizer, model) # LLMに問い合わせ

        else:
            response = "yes"  # すべて外れた場合のデフォルト応答

    except Exception as e:
        if VERBOSE:
            print(f"Error during agent execution: {e}") # エラーメッセージ

    return response  # 応答を常に返す

class Observation: # 観察クラスの定義
    def __init__(self, turnType, answers=None, keyword=None, questions=None):
        self.turnType = turnType
        self.answers = answers or [] # 回答リスト
        self.keyword = keyword
        self.questions = questions or [] # 質問リスト

# 使用例
if __name__ == "__main__":
    model, tokenizer = load_model_and_tokenizer(model_name) # モデルとトークナイザーをロード
    
    observation_object = Observation( # 観察オブジェクトを作成
        turnType="ask",
        questions=["Is it a country?"]  # 質問の例
    )
    configuration_object = {}  # 必要な設定パラメータを追加
    
    response = agent_fn(observation_object, configuration_object) # エージェント関数を呼び出す
    print(f"Agent response: {response}") # エージェントの応答を表示

    # リソースをクリーンアップ
    del model, tokenizer
    gc.collect() # ガーベジコレクションを実行
    torch.cuda.empty_cache() # CUDAキャッシュをクリア

In [ ]:
!apt install pigz pv > /dev/null # pigzとpvパッケージをインストール（出力は非表示）

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf /kaggle/working/submission.tar.gz -C /kaggle/input/gemma-2/pytorch/gemma-2-2b-it/1 . # データを圧縮し、出力される圧縮ファイルを作成するコマンド